In [1]:
from sklearn.metrics.pairwise import euclidean_distances 
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
import math
import pickle 
from sklearn.preprocessing import StandardScaler

file1 = open('X_beam.pkl', 'rb')
file2 = open('Y_beam_2.pkl', 'rb')
X1=pickle.load(file1)
file1.close()
Y=pickle.load(file2)
file2.close()
# X1= X1.astype(np.float64)  
# Y= Y.astype(np.float64)    
# Scale targets
Y1=Y[:480,:]
N, L, D = Y1.shape
Y_flat = Y1.reshape(-1, D)
scaler = StandardScaler().fit(Y_flat)
Y1 = scaler.transform(Y_flat).reshape(N, L, D)
X=X1[:480,:]

2025-07-20 16:47:04.387049: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-20 16:47:04.408004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753026424.428270  216498 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753026424.434487  216498 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753026424.450279  216498 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
Y2=scaler.transform(Y[480:,:].reshape(-1, D)).reshape(20, L, D)
Y2[0].shape

(231, 20)

In [3]:
model=tf.keras.models.load_model('saved_model_2.keras')

I0000 00:00:1753026439.156707  216498 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46478 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:3b:00.0, compute capability: 8.6


In [4]:
dtype=tf.float32


tfd = tfp.distributions 

def LogLik(Xtst,Ytst,models,cov):
    
    pred=models(tf.expand_dims(Xtst, axis=0))
    pred = tf.squeeze(pred, axis=0) 
#     scale = tf.linalg.cholesky(cov)
#     loglik=tf.reduce_sum(tfd.MultivariateNormalTriL(loc=pred,scale_tril=scale).log_prob(Ytst))
    loglik=tf.reduce_sum(tfd.Normal(loc=pred,scale=.1).log_prob(Ytst))
    return loglik

def PriorMaker(min,max):
    def prior(a,b):
        apr=tfd.Uniform(np.float32(min[0]),np.float32(max[0])).log_prob(a)
        bpr=tfd.Uniform(np.float32(min[1]),np.float32(max[1])).log_prob(b)
#         ppr=tfd.Uniform(np.float64(min[3]),np.float64(max[3])).log_prob(pre)
        # sigpr=tfd.InverseGamma(np.float64(0.001),np.float64(0.001)).log_prob(sig)
        return tf.reduce_sum(apr+bpr)
    return prior


def target(a,b,Ytst,fixed,cov,models,prior):
#     A=tf.concat([a]*2,axis=0)
#     B=tf.concat([b]*2,axis=0)
#     C=tf.concat([c]*2,axis=0)
#     D=tf.concat([d]*2,axis=0)
# First, ensure they are all [1, 1] shaped:
    a = tf.reshape(a, [1, 1])
    b = tf.reshape(b, [1, 1])
#     pre = tf.reshape(pre, [1, 1])

    # Create a single tensor from these parameters:
    params = tf.concat([a, b], axis=1)    # This should have shape [1, 2]
    batch = tf.shape(fixed)[0]
    params_tiled = tf.tile(params, [batch, 1])  # shape = [batch, 2]

    # Now concatenate it with 'fixed':
    x = tf.concat([fixed, params_tiled], axis=1)  # Concatenate along the second dimension
    lik=LogLik(x,Ytst,models,cov)
    pr=prior(a,b)
    return tf.reduce_sum(lik)+pr

In [5]:
min=np.float32([4,4])
max=np.float32([ 7,10])

prior=PriorMaker(min,max)

In [6]:
Fix=X1[480,:,:2]
Fix.shape

(231, 2)

In [7]:
Y2[0]

array([[ 4.70342832e-04,  4.70342832e-04,  4.70342832e-04, ...,
         1.09573544e+00,  1.09573544e+00,  1.09573544e+00],
       [-2.90982476e-01, -2.90982476e-01, -2.90982476e-01, ...,
         1.06380245e+00,  1.06380245e+00,  1.06380245e+00],
       [-5.36526295e-01, -5.36526295e-01, -5.36526295e-01, ...,
         1.01156997e+00,  1.01156997e+00,  1.01156997e+00],
       ...,
       [ 1.88573817e+00,  1.88573817e+00,  1.88573817e+00, ...,
        -1.22446544e+00, -1.22446544e+00, -1.22446544e+00],
       [ 1.88653175e+00,  1.88653175e+00,  1.88653175e+00, ...,
        -1.39002923e+00, -1.39002923e+00, -1.39002923e+00],
       [ 1.88646948e+00,  1.88646948e+00,  1.88646948e+00, ...,
        -1.55503556e+00, -1.55503556e+00, -1.55503556e+00]])

In [8]:
tty=model(tf.expand_dims(X1[480],axis=0))
tty[0]

<tf.Tensor: shape=(231, 20), dtype=float32, numpy=
array([[-0.02197293, -0.01667315, -0.03753584, ...,  1.09769   ,
         1.0869348 ,  1.0935947 ],
       [-0.28244504, -0.27109364, -0.28994322, ...,  1.0603195 ,
         1.0511209 ,  1.0674258 ],
       [-0.5470733 , -0.5325681 , -0.55125374, ...,  1.0101151 ,
         1.0012796 ,  1.0106198 ],
       ...,
       [ 1.885473  ,  1.8976963 ,  1.8984903 , ..., -1.2335683 ,
        -1.224186  , -1.2341461 ],
       [ 1.8729582 ,  1.8833783 ,  1.8840817 , ..., -1.4062493 ,
        -1.3919083 , -1.3950614 ],
       [ 1.8704047 ,  1.8763937 ,  1.878784  , ..., -1.5721686 ,
        -1.5592045 , -1.5540975 ]], dtype=float32)>

In [9]:
testy=Y2[0]
cov=1

Posterior=lambda a,b: target(a=a,b=b,Ytst=testy,fixed=Fix,cov=cov,models=model,prior=prior)
def BijectionMaker(min,max,log):
    bij=[]
    if log:
        for i in range(2):
            bij.append(tfb.Chain([tfb.Log(),tfb.Sigmoid(low=min[i],high=max[i])]))
    else:
        for i in range(2):
            bij.append(tfb.Shift(tf.cast(min[i],tf.float32))(tfb.Scale(tf.cast((max[i])-(min[i]),tf.float32))( tfb.Reciprocal()(
                tfb.Shift(tf.cast(1.,tf.float32))(
                  tfb.Exp()(
                    tfb.Scale(tf.cast(-1.,tf.float32))))))))
    # bij.append(tfb.Exp())
    return bij

In [10]:
tfb=tfp.bijectors
bijector=BijectionMaker(min,max,0)

def step_size_setter_fn(kernel_results, new_step_size):
    pars=kernel_results.inner_results
    pars=pars._replace(
            step_size=new_step_size)
    return kernel_results._replace(
            inner_results=pars)


def step_size_getter_fn(kernel_results):
    return tf.cast(ss, dtype)# for ss in kernel_results.inner_results.step_size]


def log_accept_prob_getter_fn(kernel_results):
    return kernel_results.inner_results.log_accept_ratio

def NutsAdaptiveKernel(FinalTarget,nburn,BIJ):
    nuts=tfp.mcmc.NoUTurnSampler(
    target_log_prob_fn=FinalTarget,
    step_size=tf.cast(0.01,tf.float32),max_tree_depth=10)
    TransNuts= tfp.mcmc.TransformedTransitionKernel(
    inner_kernel=nuts,
    bijector=BIJ)
    return tfp.mcmc.DualAveragingStepSizeAdaptation(
    TransNuts,
    num_adaptation_steps=int(nburn*0.8),
    step_size_setter_fn=lambda pkr, new_step_size: pkr._replace(
            inner_results=pkr.inner_results._replace(step_size=new_step_size)),
    step_size_getter_fn=lambda pkr: pkr.inner_results.step_size,
    log_accept_prob_getter_fn=lambda pkr: pkr.inner_results.log_accept_ratio,
    target_accept_prob=tf.cast(.9, dtype),
    decay_rate=tf.cast(.75, dtype)
)


In [11]:
nb=5000
ns=5000
# num_chains=2
kernel=NutsAdaptiveKernel(Posterior,nb,bijector)
IniState=[]
for i in range(2):
    IniState.append(tf.Variable(np.float32([[1]])))
IniState[0].assign(np.random.uniform(4,7,[1,1]))
IniState[1].assign(np.random.uniform(4,10,[1,1]))


IniState

[<tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[5.3553643]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[5.6881986]], dtype=float32)>]

In [12]:
@tf.function(experimental_compile=True)
def RunNuts(kern,init_state,nburn,nsamp):
    return tfp.mcmc.sample_chain(
    num_results=nsamp+nburn,
    num_burnin_steps=5000,
    current_state=init_state,
    trace_fn=None,
    kernel=kern
    )

states=RunNuts(kernel,IniState,nb,ns)
print(np.array(states).shape)

I0000 00:00:1753026473.775642  216498 service.cc:152] XLA service 0x558060258200 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753026473.775710  216498 service.cc:160]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2025-07-20 16:47:55.227694: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-20 16:48:03.851763: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:108] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. sanitize_seed/seed
I0000 00:00:1753026495.797634  216498 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-07-20 16:48:31.290555: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm

(2, 10000, 1, 1)


In [14]:
np.mean(states[1])

np.float32(8.929399)

In [100]:
X1[480,0,2:]

array([6.52312895, 8.98471206])